In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
%%capture
import warnings
warnings.filterwarnings('ignore')
from IPython.core.display import HTML
from jupyter_plotly_dash import JupyterDash
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go
import numpy as np
import os
from sys import path as syspath
syspath.append(os.path.expanduser("~/srdjan_functs/"))
from sys import exc_info
from caiman import movie as cmovie
import json
import pickle

In [2]:
%load_ext autoreload
%autoreload 1
%aimport Regions1, physio_def_2, numeric, Automatic1

from physio_def_2 import *
from Regions1 import Regions
from numeric import *
from Automatic1 import *

if "srdjan" not in os.getcwd():
    os.chdir(os.path.expanduser("~"))

outputStyle = {
    "color":"navy",
    "font-family":"Courier New",
    "font-size":"80%",
    }
infoStyle = {
    "font-size":"80%",
    "color":"grey",
    }
bodyStyle = {
    }

baseFig = getFigure(w=200,h=150)

## global vars
test = "srdjan" in os.getcwd()
debug = "srdjan" in os.getcwd()

if test:
    %config InlineBackend.figure_format = 'retina'

In [3]:
def tryexcept(x,default):
    try:
        return eval(x)
    except:
        return default

In [4]:
timescales = sorted([i*j for i in [1,1.5,2,2.5,3,4,5,7] for j in [.1,1,10]])[:-1]#+ sorted([i*j*60 for i in [1,1.5,2,3,5,10,30] for j in [1]])
timescales = np.array(timescales)

# tspositions = (np.log(timescales)*4).astype(int)
# tspositions -= tspositions[0]
tspositions = np.arange(len(timescales))
# plt.plot(tspositions,".-")
# mld = {.2:"ultrafast",3:"fast",600:"slow"}
# msd = {.2:"red",3:"black",600:"grey"}

# labels = {.2:"ultrafast",3:"fast",600:"slow"}

# sliderMarkers = {p:{"label":labels[ts], "style":{"color":"red"}} for p,ts in zip(tspositions,timescales) if ts in labels }
# sliderMarkers

In [21]:
folder = "/data/prototype/"

allRecs = sorted([os.path.join(folder,el) for el in os.listdir(folder) if el.endswith("lif")])

from dash import no_update

exampleLif = "/data/prototype/Experiment49e.lif"

In [32]:
app = JupyterDash(__name__,
                  width=1000,
#                   height=1000,
                 )

filterSlider = dcc.Slider(
                    id = "filter-slider",
                    min=0,
                    max=len(timescales)-1,
                    value=len(timescales)-2,
#                     marks = filterSliderMarks,
                    marks={3:{"label":"ultrafast"},
                           12:{"label":"fast"},
                           (len(timescales)-2):{"label":"detrended"},
#                            (len(timescales)-1):{"label":"raw"}
                          },
                    updatemode='drag',
                    )
SelectedRoisHidden = html.Div([
           "Selected ROIs:",
            dcc.Input(id="selected-rois",
                type="text",
                debounce=False,
                size=6,
                value="",
             ), 
        ],style={"display": "inline-box" if debug else "hidden",**infoStyle})

rangePickers = html.Div([dcc.RangeSlider(
                    id = "range-slider-%i"%i,
                    min=0,
                    max=1,
                    value=[0,1] if i==0 else [0,0],
                    updatemode='mouseup',
                    included=True,
                    step=.001
                    ) for i in range(3)])

APP_LAYOUT = [
    
    html.H1(children='CTN Analytics',style=bodyStyle),
    html.Div("how do you like the name? :-)",style=infoStyle),
    html.Br(),
    ####################################
    html.Div('Select the experiment',style={**bodyStyle, "display":"inline-box"}),
    dcc.Dropdown(options = [{"value":path,"label":path} for path in allRecs], id="filepath-dropdown", value = exampleLif if (test and exampleLif in allRecs) else None, style={**bodyStyle,"width":"70%"}),
    html.Div('Select the series',style={**bodyStyle, "display":"inline-box"}),
    dcc.Dropdown(options = [{"value":None,"label":None}], id="series-dropdown", style={**bodyStyle,"width":"70%"}),
    html.Div([
        html.Div([
            'Select filer size',
            dcc.RadioItems(options = [{"value":None,"label":None}], id="pickle-radio", labelStyle={"display":"block"})
        ],style={**bodyStyle, "display":"inline-block", "vertical-align":"text-top"}),
        html.Div(id="pickle-previews", children = ["None"],style={"display":"inline-block","padding-left":"20px","vertical-align":"text-top",}),
    ],style={"padding-top":"20px"}),
    html.Div(id="pickle-feedback",style={**outputStyle}),
    html.Div([
        html.Div([
            k if test else None,
            dcc.Graph(id=k, figure=getFigure(300,400)),
            {"spike-durations":filterSlider,"roi-selector":SelectedRoisHidden, "range-pickers":rangePickers}.get(k)
                 ],style={"max-width":"500px","max-height":"500px","border":"thin grey solid",}
                ) for k in ["roi-selector","range-pickers","spike-durations","raster", "roi-hover"]
    ],
        style={"display":"flex", "flex-wrap":"wrap","align-items":"top","flex-shrink":3}
    )
    
]

In [33]:
@app.callback(
    [Output("series-dropdown","value"),
    Output("series-dropdown","options")],
    [Input("filepath-dropdown","value")]
)
def serve_series(pathToRec):
    if pathToRec is None:
        return no_update
    if len(pathToRec)==0:
        return no_update
    analysisFolder = pathToRec+"_analysis"
    opts = [{"value":os.path.join(analysisFolder,el), "label":el} for el in os.listdir(analysisFolder) if os.path.isdir(os.path.join(analysisFolder,el))]
    global pathToSer
    pathToSer = (opts[0]["value"] if len(opts) and test else None)
    return pathToSer, opts

@app.callback(
    [Output("pickle-radio","value"),
     Output("pickle-radio","options"),
     Output("pickle-previews","children")
    ],
    [Input("series-dropdown","value")]
)
def serve_pickles(pathToSer,width=200, height=200):
    import base64
    from collections import OrderedDict 
    if pathToSer is None:
        return no_update
    if len(pathToSer)==0:
        return no_update
    preview = OrderedDict()
    options = []
    for f in sorted(os.listdir(pathToSer)):
        if f.endswith("pkl"):
            k = f.split("_")[0].replace(".","+")
            pathToRoi = os.path.join(pathToSer,f)
            options += [{ "label": k, "value": pathToRoi }]
            previewFile = pathToRoi.replace(f,".image_"+f.replace("_rois.pkl",".png"))
            if os.path.isfile(previewFile):
                encoded_image = base64.b64encode(open(previewFile, 'rb').read())
                preview[k] = html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), width="%ipx"%(width*.9),height="%ipx"%(height*.8+20))

    if len(options) and test:
        val = options[0]["value"]
    else:
        val = None
    previews = [html.Div(children=[html.Div(k,style={"padding-left":"20px"}), preview[k]], style=dict(
        height="%ipx"%height,
        width="%ipx"%width,
        display="inline-block",
        border='thin lightgrey solid'
    )) for k in preview]
    return val, options, html.Div(previews,style={"width":"%ipx"%(width*2.3)})

@app.callback(
    [Output("pickle-feedback","children"),
     Output("roi-selector","figure"),
    ],
    [Input("pickle-radio","value")]
)
def import_pickle(path):
    if path is None:
        return no_update
    if len(path)==0:
        return no_update
    global regions
    with open(path,"rb") as f:
        regions = pickle.load(f)
    regions.update()
    feedback = [
        "Regions imported successfully.",
        "Original movie:",
        html.Div(children=[
            "dimension (T,X,Y): (%i, %i, %i)"%((len(regions.time), )+regions.image.shape),
            html.Br(),
            "duration (h:m:s): "+str(pd.Timedelta(regions.time.max(), unit="s")).split()[-1].split(".")[0]],
            style={'padding-left': '30px'}
        ),
        "Number of ROIs: %i"%len(regions.df),
    ]
    feedback = sum([[el, html.Br()] for el in feedback],[])
    roisImage = showRoisOnly(regions,indices=regions.df.index, im=regions.statImages[regions.mode])
    return feedback,roisImage

@app.callback(
    Output("spike-durations", "figure"),
    [Input("filter-slider", "value")])
def plotPeakStats_callback(i):
    timeScale = timescales[i]
    global regions
    k = "%g"%timeScale
    try:
        regions.peaks[k]
    except:
        regions.calc_peaks(timeScale)
    trfig = regions.show_scatter_peaks(timeScale)
    return trfig

In [34]:
@app.callback(
    Output("selected-rois", "value"),
    [Input("roi-selector", "selectedData")],
    )
def showSelected(selData):
    if selData is None:
        return "all"
    return ",".join(p["hovertext"] for p in selData["points"])

In [59]:
@app.callback(
    Output("range-picker", "figure"),
    [Input("selected-rois", "value")],
    )
def showTrace(selData,showFreq = 2):
    global regions
    if selData is None:
        return baseFig
    if selData=="all":
        indices = regions.df.index
    else:
        indices = np.array(selData.split(",")).astype(int)
    nRebin = int(regions.Freq/showFreq)
    t = regions.time
    y = np.sum([regions.df.loc[i,"trace"]*regions.df.loc[i,"size"] for i in indices],axis=0)/regions.df["size"].sum()
    if nRebin>1:
        y = rebin(y, nRebin)
        t = rebin(t, nRebin)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=t,y=y))
    fig.update_layout({
        "plot_bgcolor":"white",
        "width":600,
        "height":400,
#         "dragmode":'drawrect',
    })
#     dragmode=,
#     newshape_line_color='cyan',
#     title_text='Draw a path to separate versicolor and virginica'
    return fig#.show(config={'modeBarButtonsToAdd':['drawline',
#                                     'drawopenpath',
#                                     'drawclosedpath',
#                                     'drawcircle',
#                                     'drawrect',
#                                     'eraseshape'
#                                    ]})

In [60]:
showTrace("all")

In [35]:
app.layout = html.Div(children=APP_LAYOUT,
                      style={"family":"Arial"}
                     )
app

In [26]:
f = fov_trace(regions)

AttributeError: 'Regions' object has no attribute 'fov_trace'

In [445]:
f

In [446]:
f.update_layout({"height":200})

In [400]:
regions.time.max()

5753.02

In [28]:
# app.layout = html.Div(children=APP_LAYOUT)
# app._repr_html_() 
# link2app = "https://ctn.physiologie.meduniwien.ac.at"+app.get_app_root_url()
# HTML(f'open the following link in a different tab (do not close this tab!): <a href="{link2app}">{link2app}</a>')

In [384]:
regions.show_scatter_peaks(3)

[autoreload of Regions1 failed: Traceback (most recent call last):
  File "/opt/tljh/user/envs/physio/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/opt/tljh/user/envs/physio/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/opt/tljh/user/envs/physio/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/opt/tljh/user/envs/physio/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 724, in exec_module
  File "<frozen importlib._bootstrap_external>", line 860, in get_code
  File "<frozen importlib._bootstrap_external>", line 791, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/home/jupy

AttributeError: 'Regions' object has no attribute 'peaks'